In [1]:
import ee
ee.Authenticate()  # Run this in command line or Jupyter notebook
ee.Initialize()

In [2]:
import ee
import geojson
import geopandas as gpd
from shapely.geometry import Polygon
import os

In [4]:
import ee

# Initialize the Earth Engine API
ee.Initialize()

def export_tif_to_drive(city_name, center_lat, center_lng, half_side_km=20, scale=10):
    """
    Export Open Buildings data for a given city and two time ranges (2016-2017 and 2023-2024).
    
    Parameters:
        city_name (str): Name of the city.
        center_lat (float): Latitude of the city center.
        center_lng (float): Longitude of the city center.
        half_side_km (float, optional): Half of the side length of the square in km. Default is 20.
        scale (int, optional): Export resolution in meters. Default is 10.
    """
    # Convert km to degrees (approximate)
    half_side_deg = half_side_km / 111.32

    # Calculate coordinates of the square's corners
    coords = [
        [center_lng - half_side_deg, center_lat - half_side_deg],  # Bottom-left
        [center_lng + half_side_deg, center_lat - half_side_deg],  # Bottom-right
        [center_lng + half_side_deg, center_lat + half_side_deg],  # Top-right
        [center_lng - half_side_deg, center_lat + half_side_deg],  # Top-left
        [center_lng - half_side_deg, center_lat - half_side_deg]   # Closing the polygon
    ]

    region = ee.Geometry.Polygon([coords])

    # Get Open Buildings dataset
    buildings = ee.ImageCollection("GOOGLE/Research/open-buildings-temporal/v1")
    buildings_filtered = buildings.filterBounds(region)

    # Filter by years
    buildings_2016 = buildings_filtered.filter(ee.Filter.calendarRange(2016, 2017, 'year'))
    buildings_2023 = buildings_filtered.filter(ee.Filter.calendarRange(2023, 2024, 'year'))

    # Export for 2016
    task_2016 = ee.batch.Export.image.toDrive(
        image=buildings_2016.mosaic(),
        description=f'{city_name}_2016',
        scale=scale,
        region=region,
        fileFormat='GeoTIFF'
    )

    # Export for 2023
    task_2023 = ee.batch.Export.image.toDrive(
        image=buildings_2023.mosaic(),
        description=f'{city_name}_2023',
        scale=scale,
        region=region,
        fileFormat='GeoTIFF'
    )

    # Start export tasks
    task_2016.start()
    task_2023.start()

    print(f"Export tasks for {city_name} started. Check your Google Drive after processing.")

In [5]:
# usage: 纬度+经度 和google earth顺序一致
#export_tif_to_drive("Abidjan", 5.3252258, -4.019603)
#export_tif_to_drive("Antananarivo", -18.9155584, 47.5216452)
#export_tif_to_drive("Yaounde", 3.8614776, 11.5191364)
#export_tif_to_drive("Lusaka", -15.4191874, 28.3054202)
#export_tif_to_drive("Buenos Aires", -34.609504, -58.389111)
#export_tif_to_drive("Hanoi",21.0283,105.8542)
#export_tif_to_drive("Medan",3.5894,98.6739)
#export_tif_to_drive("Monterrey",25.6667,-100.3)

Export tasks for Monterrey started. Check your Google Drive after processing.


In [6]:
import pandas as pd
df= pd.read_csv("CitiesDB_new.csv")
# 遍历城市，每10个为一组
batch_size = 10
cities = df[["Name", "Cy(lat)", "Cx(lon)"]]

# 主循环
while True:
    try:
        user_input = input("\n请输入要处理的城市编号范围，例如 0 9 表示处理第0到第9个城市，输入 q 退出：\n")
        if user_input.strip().lower() == 'q':
            print("✅ 用户退出。")
            break

        start_idx, end_idx = map(int, user_input.strip().split())
        if start_idx < 0 or end_idx >= len(cities) or start_idx > end_idx:
            print(f"❌ 范围不合法。请输入从 0 到 {len(cities) - 1} 的有效编号，且起始值 ≤ 结束值。")
            continue

        batch = cities.iloc[start_idx:end_idx+1]

        print(f"\n🗂️ 即将处理城市编号 {start_idx} 到 {end_idx}（共 {len(batch)} 个）：\n")
        for idx, row in batch.iterrows():
            city_name = row["Name"]
            lat = row["Cy(lat)"]
            lon = row["Cx(lon)"]
            print(f"  [{idx}] {city_name} | lat: {lat}, lon: {lon}")
        
        confirm = input("\n是否确认开始处理？输入 y 开始，其他键取消本批处理：")
        if confirm.strip().lower() != 'y':
            print("⏸️ 取消当前批处理。")
            continue

        for idx, row in batch.iterrows():
            city_name = row["Name"]
            lat = row["Cy(lat)"]
            lon = row["Cx(lon)"]
            export_tif_to_drive(city_name, lat, lon)

        print(f"\n✅ 已完成第 {start_idx} 到 {end_idx} 个城市的处理。\n")

    except Exception as e:
        print(f"❌ 输入错误，请输入两个整数，例如：0 9。错误信息：{e}")


请输入要处理的城市编号范围，例如 0 9 表示处理第0到第9个城市，输入 q 退出：
 56 57



🗂️ 即将处理城市编号 56 到 57（共 2 个）：

  [56] Bangalore | lat: 12.972095, lon: 77.594172
  [57] Ho Chi Minh City | lat: 10.778004, lon: 106.696322



是否确认开始处理？输入 y 开始，其他键取消本批处理： 


⏸️ 取消当前批处理。



请输入要处理的城市编号范围，例如 0 9 表示处理第0到第9个城市，输入 q 退出：
 54 54


...


🗂️ 即将处理城市编号 55 到 55（共 1 个）：

  [55] Karachi | lat: 24.849182, lon: 67.002087



是否确认开始处理？输入 y 开始，其他键取消本批处理： y


Export tasks for Karachi started. Check your Google Drive after processing.

✅ 已完成第 55 到 55 个城市的处理。




请输入要处理的城市编号范围，例如 0 9 表示处理第0到第9个城市，输入 q 退出：
 q


✅ 用户退出。


In [6]:
import rasterio
import numpy as np

def compare_tif_images(image_path1, image_path2):
    with rasterio.open(image_path1) as src1, rasterio.open(image_path2) as src2:
        # Read the data
        data1 = src1.read()
        data2 = src2.read()
        
        # Check if the shapes are the same (bands, height, width)
        if data1.shape != data2.shape:
            print("Images have different shapes.")
            return False

        # Compare pixel values: if all values are equal, the images are identical
        if np.array_equal(data1, data2):
            print("The images are identical.")
            return True
        else:
            print("The images are different.")
            return False

        # Optionally, compare metadata (CRS, transform, etc.)
        if src1.meta != src2.meta:
            print("Metadata is different.")
            return False
        else:
            print("Metadata is identical.")
            return True

# Example usage
image_path1 = "Abidjan_2023.tif"
image_path2 = "Abidjan_OpenBuildings_2023.tif"
compare_tif_images(image_path1, image_path2)


The images are identical.


True